# 한국어 텍스트 감정 분석 - PyTorch 모델 학습

이 노트북은 전처리된 데이터를 사용하여 PyTorch 기반의 BERT/RoBERTa 모델을 학습합니다.

## 주요 기능
- PyTorch 네이티브 학습 루프 (Transformers Trainer 대신)
- LoRA 파인튜닝
- WandB 실험 추적
- 혼합 정밀도 학습 (Mixed Precision)
- 조기 종료 및 모델 체크포인팅
- 테스트 데이터 추론 및 제출 파일 생성


In [1]:
# Library Import
import os
import math
import warnings
from collections import Counter
import platform
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from transformers import get_linear_schedule_with_warmup
from torch.cuda.amp import autocast, GradScaler
from tqdm.auto import tqdm
import wandb

# Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    set_seed,
    AutoModelForMaskedLM
)

# PEFT for LoRA
from peft import get_peft_model, LoraConfig, TaskType

# Sklearn
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 경고 메시지 필터링
warnings.filterwarnings("ignore")

print("✅ 라이브러리 임포트 완료")


/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotat

✅ 라이브러리 임포트 완료


In [2]:
EXPERIMENT_NAME = "DAPT_30_epochs_augX3"
PROJECT_NAME = f"[domain_project]_Experiment_DAPT"
MODEL_NAME = "kykim_bert-kor-base"

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
# 환경 설정
RANDOM_STATE = 42
set_seed(RANDOM_STATE)

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("⚠️  CUDA 사용 불가 - CPU로 훈련 진행")



디바이스: cuda
GPU 개수: 1
   GPU 0: Tesla V100-SXM2-32GB


In [4]:
from dotenv import load_dotenv
import os

# 이 함수가 .env 파일을 읽어서 환경 변수로 로드합니다.
load_dotenv()


True

In [5]:
molels = ["beomi_kcbert-base", "klue_roberta-base", "klue_bert-base","kykim_bert-kor-base", "monologg_koelectra-base-v3-discriminator"]

In [6]:
# 하이퍼파라미터 설정

class Config:
    # 모델 설정
    model_name = MODEL_NAME
    base_models_path = "/data/ephemeral/home/code/basemodels/"
    local_model_path = base_models_path + MODEL_NAME
    num_classes = 4
    max_length = 128

    save_best_model_path = "./best_unsupervised_model"
    
    # 훈련 설정
    unsupervised_batch_size = 196
    #batch_size = 256

    DAPT_num_epochs = 20
    TAPT_num_epochs = 30


    learning_rate = 2e-5
    weight_decay = 0.01
    warmup_steps = 500
    
    # 기타 설정
    gradient_accumulation_steps = 1
    max_grad_norm = 1.0
    early_stopping_patience = 3
    save_best_model = True
    
    # WandB 설정
    use_wandb = True
    project_name = PROJECT_NAME
    run_name = f"{EXPERIMENT_NAME}-training"

config = Config()
print("✅ 하이퍼파라미터 설정 완료")


✅ 하이퍼파라미터 설정 완료


In [7]:
# 데이터 로드
print("전처리된 데이터 로드 중...")

# 훈련 데이터 로드
train_df = pd.read_csv("/data/ephemeral/home/code/data/train_processed_augX3_newly_gen_added.csv")
print(f"훈련 데이터: {len(train_df):,}개")

# 검증 데이터 로드
val_df = pd.read_csv("/data/ephemeral/home/code/data/val_processed_augX3_newly_gen_added.csv")
print(f"검증 데이터: {len(val_df):,}개")

test_df = pd.read_csv("data/test_processed.csv")
print(f"테스트 데이터: {len(test_df):,}개")

# 라벨 매핑
LABEL_MAPPING = {0: "강한 부정", 1: "약한 부정", 2: "약한 긍정", 3: "강한 긍정"}

print("✅ 데이터 로드 완료")


전처리된 데이터 로드 중...
훈련 데이터: 571,014개
검증 데이터: 63,446개
테스트 데이터: 59,928개
✅ 데이터 로드 완료


In [8]:
train_df.columns

Index(['ID', 'review', 'label', 'type'], dtype='object')

In [ ]:
class UnlabeledDataset(Dataset):
    """
    라벨이 없는 데이터셋 클래스 (ELECTRA RTD 작업 지원)
    """
    def __init__(self, texts, tokenizer, max_length, mask_probability=0.15):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mask_probability = mask_probability
        self.mask_token_id = tokenizer.mask_token_id
        self.vocab_size = tokenizer.vocab_size
    
    def __len__(self):
        return len(self.texts)
    
    def _apply_electra_masking(self, input_ids, attention_mask):
        """ELECTRA 스타일 마스킹 적용 (Generator용)"""
        labels = input_ids.clone()
        
        # 패딩 토큰이 아닌 위치만 선택
        non_padding_mask = attention_mask.bool()
        
        # 마스킹할 토큰 선택 (패딩 토큰 제외)
        candidate_indices = torch.where(non_padding_mask)[0]
        
        if len(candidate_indices) == 0:
            return input_ids, labels
        
        # 마스킹할 토큰 수 계산
        num_tokens_to_mask = max(1, int(len(candidate_indices) * self.mask_probability))
        
        # 랜덤하게 토큰 선택
        selected_indices = torch.randperm(len(candidate_indices))[:num_tokens_to_mask]
        selected_positions = candidate_indices[selected_indices]
        
        # 선택된 토큰에 대해 마스킹 적용
        for pos in selected_positions:
            if torch.rand(1) < 1:
                input_ids[pos] = self.mask_token_id
        
        return input_ids, labels
    
    def __getitem__(self, idx):
        """인덱스에 해당하는 샘플 반환 (ELECTRA RTD 작업용)"""
        # 텍스트 추출 - 문자열로 변환 보장
        if hasattr(self.texts, 'iloc'):
            text = str(self.texts.iloc[idx])  # pandas Series인 경우
        else:
            text = str(self.texts[idx])  # list나 다른 타입인 경우
        
        # None이나 NaN 값 처리
        if text == 'None' or text == 'nan' or text == '':
            text = "빈 텍스트입니다."  # 기본값 설정
        
        # 텍스트 토크나이징
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        
        # ELECTRA 마스킹 적용 (Generator용)
        masked_input_ids, mlm_labels = self._apply_electra_masking(input_ids, attention_mask)
        
        return {
            'input_ids': input_ids,  # 원본 토큰 (Discriminator용)
            'masked_input_ids': masked_input_ids,  # 마스킹된 토큰 (Generator용)
            'attention_mask': attention_mask,
            'mlm_labels': mlm_labels  # Generator 학습을 위한 라벨
        }

In [10]:
# 평가 메트릭 함수
def compute_metrics(predictions, labels):
    """
    모델 평가 메트릭 계산 함수
    """
    # 예측값에서 가장 높은 확률의 클래스 선택
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    
    return {
        "accuracy": accuracy,
        "f1": f1,
    }

print("✅ 평가 메트릭 함수 정의 완료")


✅ 평가 메트릭 함수 정의 완료


In [ ]:
def model_load_from_local(local_model_path : str = None, model_name : str = None):
    # 1. 토크나이저 로드
    # 절대 경로를 사용하여 로컬 모델 로드
    print("로컬 경로에서 토크나이저 로딩 중...")

    # 경로가 존재하는지 확인
    if not os.path.exists(local_model_path):
        print(f"❌ 경로가 존재하지 않습니다: {local_model_path}")
        return None, None, None
    else:
        print(f"✅ 경로 확인됨: {local_model_path}")

    tokenizer = AutoTokenizer.from_pretrained(local_model_path)

    # 2. ELECTRA 모델 로드 (Generator + Discriminator)
    print("로컬 경로에서 ELECTRA 모델 로딩 중...")
    
    # Generator (MLM용)
    generator = AutoModelForMaskedLM.from_pretrained(local_model_path)
    
    # Discriminator (RTD용) - Generator와 동일한 구조이지만 RTD 헤드 사용
    discriminator = AutoModelForMaskedLM.from_pretrained(local_model_path)
    
    # Discriminator의 MLM 헤드를 RTD 헤드로 교체
    from transformers import ElectraForPreTraining
    discriminator = ElectraForPreTraining.from_pretrained(local_model_path)

    print("✅ 로컬 스냅샷에서 ELECTRA 모델과 토크나이저 로딩 성공!")

    # --- 이제 평소처럼 모델을 사용할 수 있습니다 ---
    inputs = tokenizer("이 영화 정말 재미있네요!", return_tensors="pt")
    outputs = generator(**inputs)
    print("Generator outputs shape:", outputs.logits.shape)

    return generator, discriminator, tokenizer

In [ ]:
generator, discriminator, tokenizer = model_load_from_local(config.local_model_path, config.model_name)

로컬 경로에서 토크나이저 로딩 중...
✅ 경로 확인됨: /data/ephemeral/home/code/basemodels/kykim_bert-kor-base
로컬 경로에서 모델 로딩 중...


Some weights of the model checkpoint at /data/ephemeral/home/code/basemodels/kykim_bert-kor-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✅ 로컬 스냅샷에서 모델과 토크나이저 로딩 성공!
tensor([[[ -8.7732,  -5.3598,  -4.4707,  ...,  -5.5349,  -4.1685,  -6.2984],
         [-13.5281,  -7.3256,  -5.6978,  ...,  -4.6070,  -7.2017, -13.3684],
         [-13.5850, -10.6456,  -6.2533,  ...,  -5.0207,  -6.9485, -10.5358],
         ...,
         [-13.1823,  -8.8666,  -4.9712,  ..., -10.3408,  -7.1187, -11.3603],
         [-11.3309,  -6.7285,  -2.5937,  ...,  -6.8013,  -4.9377, -10.5218],
         [ -9.3873,  -4.1437,  -6.7594,  ...,  -3.4999,  -1.9037,  -9.5742]]],
       grad_fn=<ViewBackward0>)


In [ ]:
# 모델을 디바이스로 이동
generator = generator.to(device)
discriminator = discriminator.to(device)

print("✅ ELECTRA 모델 및 토크나이저 로드 완료")


✅ 모델 및 토크나이저 로드 완료


In [14]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [15]:
unlabeled_df = pd.DataFrame(pd.concat([
    train_df["review"], 
    val_df["review"], 
    test_df["review"]
], ignore_index=True))

original_df = pd.DataFrame(pd.concat([
    train_df[train_df["type"] == "original"]["review"], 
    val_df[val_df["type"] == "original"]["review"], 
    test_df["review"]
], ignore_index=True))

print(unlabeled_df)
print(len(original_df))


                                                   review
0                          이런 애매한 끝맺음은 좀 아니지 않나 싶네. ㅡ,. ㅡ
1       와, 오랜만에 이 영화 다시 봤는데 정말이지. 편집은 무슨 정신으로 한 건지 도저히...
2       캐릭터들도 하나같이 너무 매력적이어서 보는 내내 미소가 끊이질 않았어. 특히 조카가...
3           나도 소름 끼치면서 잘봣는데! 나이트샤말론감독 영화는 다른영화들과 좀 다른듯!멋짐
4       두 사람의 케미가 정말 최고였고, 보는 내내 입가에 미소가 떠나지 않았어요. 특히 ...
...                                                   ...
694383  여자 주인공은 예쁜데 잠깐 나오고 전체적으로는 좋은 내용이지만 세부적인 내용들이 많...
694384                기대 안하고 봤는데 오랜만에 보는 정말 좋은 코미디 영화였습니다
694385  서론이 너무길고 별거아닌거에 나레이션을 너무많이깔아서 지루함.. 스토리는 별거아닌데...
694386                                   고급스럽게 포장된 고급 영화.
694387                             난이런밋밋한영화가별로라서..살짝지루했음.

[694388 rows x 1 columns]
196381


In [16]:
# 데이터셋 및 데이터로더 생성
print("데이터셋 및 데이터로더 생성 중...")

original_dataset = UnlabeledDataset(
    original_df["review"],
    tokenizer,
    config.max_length,
)

unlabeled_dataset = UnlabeledDataset(
    unlabeled_df["review"],
    tokenizer,
    config.max_length,
)

unlabeled_dataloader = DataLoader(
    unlabeled_dataset,
    batch_size=config.unsupervised_batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

original_dataloader = DataLoader(
    original_dataset,
    batch_size=config.unsupervised_batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"Unlabeled 데이터: {len(unlabeled_dataset):,}개")
print(f"Original 데이터: {len(original_dataset):,}개")
print("✅ 데이터셋 및 데이터로더 생성 완료")


데이터셋 및 데이터로더 생성 중...
Unlabeled 데이터: 694,388개
Original 데이터: 196,381개
✅ 데이터셋 및 데이터로더 생성 완료


In [ ]:
# 옵티마이저 및 스케줄러 설정
print("옵티마이저 및 스케줄러 설정 중...")

# 옵티마이저 설정 (Generator + Discriminator)
optimizer = AdamW(
    list(generator.parameters()) + list(discriminator.parameters()),
    lr=config.learning_rate,
    weight_decay=config.weight_decay
)

# 전체 훈련 스텝 계산
DAPT_total_steps = len(unlabeled_dataloader) * config.DAPT_num_epochs // config.gradient_accumulation_steps
TAPT_total_steps = len(original_dataloader) * config.TAPT_num_epochs // config.gradient_accumulation_steps

# 스케줄러 설정 (warmup + linear decay)
DAPT_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config.warmup_steps,
    num_training_steps=DAPT_total_steps
)
TAPT_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config.warmup_steps,
    num_training_steps=TAPT_total_steps
)

# 혼합 정밀도 스케일러
scaler = GradScaler() if torch.cuda.is_available() else None

print(f"DAPT 총 훈련 스텝: {DAPT_total_steps:,}")
print(f"TAPT 총 훈련 스텝: {TAPT_total_steps:,}")
print(f"DAPT 워밍업 스텝: {config.warmup_steps:,}")
print(f"TAPT 워밍업 스텝: {config.warmup_steps:,}")
print("✅ 옵티마이저 및 스케줄러 설정 완료")


옵티마이저 및 스케줄러 설정 중...
DAPT 총 훈련 스텝: 70,860
TAPT 총 훈련 스텝: 30,060
DAPT 워밍업 스텝: 500
TAPT 워밍업 스텝: 500
✅ 옵티마이저 및 스케줄러 설정 완료


In [18]:
# WandB 초기화
if config.use_wandb:
    wandb.init(
        project=config.project_name,
        name=config.run_name,
        config={
            "model_name": config.model_name,
            "DAPT_num_epochs": config.DAPT_num_epochs,
            "TAPT_num_epochs": config.TAPT_num_epochs,
            "batch_size": config.unsupervised_batch_size,
            "learning_rate": config.learning_rate,
            "max_length": config.max_length,
            "num_classes": config.num_classes,
            "warmup_steps": config.warmup_steps,
            "weight_decay": config.weight_decay,
            # "lora_r": config.lora_r,
            # "lora_alpha": config.lora_alpha,
            # "lora_dropout": config.lora_dropout,
            "random_seed": RANDOM_STATE
        }
    )
    print("✅ WandB 초기화 완료")
else:
    print("⚠️  WandB 사용 안함")


wandb: Currently logged in as: qkdwodus777 (qkdwodus777-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ WandB 초기화 완료


In [ ]:
def unsupervie_train_epoch(generator, discriminator, dataloader, optimizer, scheduler, scaler, device, config):
    """한 에포크 훈련 (ELECTRA 비지도 학습)"""
    generator.train()
    discriminator.train()
    total_loss = 0
    total_mlm_loss = 0
    total_rtd_loss = 0
    num_batches = 0
    
    progress_bar = tqdm(dataloader, desc="ELECTRA Unsupervised Training")
    
    for step, batch in enumerate(progress_bar):
        input_ids = batch["input_ids"].to(device)  # 원본 토큰
        masked_input_ids = batch["masked_input_ids"].to(device)  # 마스킹된 토큰
        attention_mask = batch["attention_mask"].to(device)
        mlm_labels = batch["mlm_labels"].to(device)  # Generator용 라벨
        
        # 그래디언트 초기화
        optimizer.zero_grad()
        
        # 혼합 정밀도 학습
        if scaler is not None:
            with autocast():
                # 1. Generator 학습 (MLM)
                gen_outputs = generator(
                    input_ids=masked_input_ids,
                    attention_mask=attention_mask,
                    labels=mlm_labels
                )
                mlm_loss = gen_outputs.loss
                
                # 2. Generator로 대체 토큰 생성
                with torch.no_grad():
                    gen_logits = generator(
                        input_ids=masked_input_ids,
                        attention_mask=attention_mask
                    ).logits
                    generated_tokens = torch.argmax(gen_logits, dim=-1)
                
                # 3. RTD 라벨 생성 (원본과 다른 토큰 위치)
                rtd_labels = (input_ids != generated_tokens).float()
                
                # 4. Discriminator 학습 (RTD)
                disc_outputs = discriminator(
                    input_ids=generated_tokens,
                    attention_mask=attention_mask,
                    labels=rtd_labels
                )
                rtd_loss = disc_outputs.loss
                
                # 5. 전체 손실 (Generator + Discriminator)
                total_batch_loss = mlm_loss + rtd_loss
                
            # 스케일된 그래디언트 계산
            scaler.scale(total_batch_loss).backward()
            
            # 그래디언트 클리핑
            if config.max_grad_norm > 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(list(generator.parameters()) + list(discriminator.parameters()), config.max_grad_norm)
            
            # 옵티마이저 스텝
            scaler.step(optimizer)
            scaler.update()
            
        else:
            # 일반 정밀도 학습
            # 1. Generator 학습 (MLM)
            gen_outputs = generator(
                input_ids=masked_input_ids,
                attention_mask=attention_mask,
                labels=mlm_labels
            )
            mlm_loss = gen_outputs.loss
            
            # 2. Generator로 대체 토큰 생성
            with torch.no_grad():
                gen_logits = generator(
                    input_ids=masked_input_ids,
                    attention_mask=attention_mask
                ).logits
                generated_tokens = torch.argmax(gen_logits, dim=-1)
            
            # 3. RTD 라벨 생성 (원본과 다른 토큰 위치)
            rtd_labels = (input_ids != generated_tokens).float()
            
            # 4. Discriminator 학습 (RTD)
            disc_outputs = discriminator(
                input_ids=generated_tokens,
                attention_mask=attention_mask,
                labels=rtd_labels
            )
            rtd_loss = disc_outputs.loss
            
            # 5. 전체 손실 (Generator + Discriminator)
            total_batch_loss = mlm_loss + rtd_loss
            
            # 역전파
            total_batch_loss.backward()
            
            # 그래디언트 클리핑
            if config.max_grad_norm > 0:
                torch.nn.utils.clip_grad_norm_(list(generator.parameters()) + list(discriminator.parameters()), config.max_grad_norm)
            
            # 옵티마이저 스텝
            optimizer.step()
        
        # 스케줄러 스텝
        if scheduler is not None:
            scheduler.step()
        
        # 손실 누적
        total_loss += total_batch_loss.item()
        total_mlm_loss += mlm_loss.item()
        total_rtd_loss += rtd_loss.item()
        num_batches += 1
        
        # 진행률 표시 업데이트
        avg_loss = total_loss / num_batches
        avg_mlm_loss = total_mlm_loss / num_batches
        avg_rtd_loss = total_rtd_loss / num_batches
        progress_bar.set_postfix({
            'total_loss': f'{avg_loss:.4f}',
            'mlm_loss': f'{avg_mlm_loss:.4f}',
            'rtd_loss': f'{avg_rtd_loss:.4f}',
            'lr': f'{optimizer.param_groups[0]["lr"]:.2e}'
        })
        
        # 그래디언트 누적 (선택사항)
        if config.gradient_accumulation_steps > 1:
            if (step + 1) % config.gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
                if scheduler is not None:
                    scheduler.step()
    
    # 평균 손실 계산
    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    avg_mlm_loss = total_mlm_loss / num_batches if num_batches > 0 else 0
    avg_rtd_loss = total_rtd_loss / num_batches if num_batches > 0 else 0
    
    return avg_loss, avg_mlm_loss, avg_rtd_loss

In [ ]:
def DAPT_training_loop(generator, discriminator, unlabeled_dataloader, optimizer, scheduler, scaler, device, config):
    """ELECTRA 비지도 학습 메인 루프"""
    
    # 초기화
    best_unsup_loss = float('inf')
    patience_counter = 0

    print("🚀 ELECTRA DAPT 학습 시작...")
    print(f"총 에포크: {config.DAPT_num_epochs}")
    print(f"조기 종료 인내심: {config.early_stopping_patience}")
    
        
    for epoch in range(config.DAPT_num_epochs):
        print(f"\n=== Epoch {epoch+1}/{config.DAPT_num_epochs} ===")
        
        # ELECTRA 비지도 학습 에포크
        total_loss, mlm_loss, rtd_loss = unsupervie_train_epoch(
            generator=generator,
            discriminator=discriminator,
            dataloader=unlabeled_dataloader,
            optimizer=optimizer,
            scheduler=scheduler,
            scaler=scaler,
            device=device,
            config=config
        )
        
        print(f"Total Loss: {total_loss:.4f}")
        print(f"MLM Loss: {mlm_loss:.4f}")
        print(f"RTD Loss: {rtd_loss:.4f}")
        
        # 모델 저장 조건 (손실이 개선되었을 때)
        if config.save_best_model and total_loss < best_unsup_loss:
            best_unsup_loss = total_loss
            patience_counter = 0
            
            # 모델 저장
            os.makedirs(config.save_best_model_path, exist_ok=True)
            generator.save_pretrained(f"./{config.save_best_model_path}/DAPT_{MODEL_NAME}_augX3_best_generator_1028")
            discriminator.save_pretrained(f"./{config.save_best_model_path}/DAPT_{MODEL_NAME}_augX3_best_discriminator_1028")
            tokenizer.save_pretrained(f"./{config.save_best_model_path}/DAPT_{MODEL_NAME}_augX3_best_generator_1028")
            print(f"✅ 최고 성능 ELECTRA 모델 저장 (Total Loss: {best_unsup_loss:.4f})")
            
        else:
            patience_counter += 1
            print(f"⏳ 조기 종료 카운터: {patience_counter}/{config.early_stopping_patience}")
        
        # WandB 로깅 (선택사항)
        if config.use_wandb:
            wandb.log({
                "epoch": epoch + 1,
                "total_loss": total_loss,
                "mlm_loss": mlm_loss,
                "rtd_loss": rtd_loss,
                "best_unsupervised_loss": best_unsup_loss,
                "learning_rate": optimizer.param_groups[0]['lr']
            })
        
        # 조기 종료 체크
        if patience_counter >= config.early_stopping_patience:
            print(f"🛑 조기 종료: {config.early_stopping_patience} 에포크 동안 개선 없음")
            break
    
    print(f"\n🎯 ELECTRA 비지도 학습 완료!")
    print(f"최고 Total Loss: {best_unsup_loss:.4f}")
    
    return best_unsup_loss

In [ ]:
def TAPT_training_loop(generator, discriminator, unlabeled_dataloader, optimizer, scheduler, scaler, device, config):
    """ELECTRA 비지도 학습 메인 루프"""
    
    # 초기화
    best_unsup_loss = float('inf')
    patience_counter = 0

    print("🚀 ELECTRA TAPT 학습 시작...")
    print(f"총 에포크: {config.TAPT_num_epochs}")
    print(f"조기 종료 인내심: {config.early_stopping_patience}")
    
        
    for epoch in range(config.TAPT_num_epochs):
        print(f"\n=== Epoch {epoch+1}/{config.TAPT_num_epochs} ===")
        
        # ELECTRA 비지도 학습 에포크
        total_loss, mlm_loss, rtd_loss = unsupervie_train_epoch(
            generator=generator,
            discriminator=discriminator,
            dataloader=unlabeled_dataloader,
            optimizer=optimizer,
            scheduler=scheduler,
            scaler=scaler,
            device=device,
            config=config
        )
        
        print(f"Total Loss: {total_loss:.4f}")
        print(f"MLM Loss: {mlm_loss:.4f}")
        print(f"RTD Loss: {rtd_loss:.4f}")
        
        # 모델 저장 조건 (손실이 개선되었을 때)
        if config.save_best_model and total_loss < best_unsup_loss:
            best_unsup_loss = total_loss
            patience_counter = 0
            
            # 모델 저장
            os.makedirs(config.save_best_model_path, exist_ok=True)
            generator.save_pretrained(f"./{config.save_best_model_path}/TAPT_{MODEL_NAME}_augX3_best_generator_1026")
            discriminator.save_pretrained(f"./{config.save_best_model_path}/TAPT_{MODEL_NAME}_augX3_best_discriminator_1026")
            tokenizer.save_pretrained(f"./{config.save_best_model_path}/TAPT_{MODEL_NAME}_augX3_best_generator_1026")
            print(f"✅ 최고 성능 ELECTRA 모델 저장 (Total Loss: {best_unsup_loss:.4f})")
            
        else:
            patience_counter += 1
            print(f"⏳ 조기 종료 카운터: {patience_counter}/{config.early_stopping_patience}")
        
        # WandB 로깅 (선택사항)
        if config.use_wandb:
            wandb.log({
                "epoch": epoch + 1,
                "total_loss": total_loss,
                "mlm_loss": mlm_loss,
                "rtd_loss": rtd_loss,
                "best_unsupervised_loss": best_unsup_loss,
                "learning_rate": optimizer.param_groups[0]['lr']
            })
        
        # 조기 종료 체크
        if patience_counter >= config.early_stopping_patience:
            print(f"🛑 조기 종료: {config.early_stopping_patience} 에포크 동안 개선 없음")
            break
    
    print(f"\n🎯 ELECTRA 비지도 학습 완료!")
    print(f"최고 Total Loss: {best_unsup_loss:.4f}")
    
    return best_unsup_loss

In [ ]:
best_loss_stage_DAPT = DAPT_training_loop(
    generator=generator,
    discriminator=discriminator,
    unlabeled_dataloader=unlabeled_dataloader,
    optimizer=optimizer,
    scheduler=DAPT_scheduler,
    scaler=scaler,
    device=device,
    config=config,
)

🚀 DAPT 학습 시작...
총 에포크: 20
조기 종료 인내심: 3

=== Epoch 1/20 ===


Unsupervised Training:   0%|          | 0/3543 [00:00<?, ?it/s]

In [ ]:
best_loss_stage_2 = TAPT_training_loop(
    generator=generator,
    discriminator=discriminator,
    unlabeled_dataloader=original_dataloader,
    optimizer=optimizer,
    scheduler=TAPT_scheduler,
    scaler=scaler,
    device=device,
    config=config,
)

In [ ]:
# # 추론 실행
# print("추론 실행 중...")
# inference_model.eval()
# predictions = []

# with torch.no_grad():
#     for batch in tqdm(test_dataloader, desc="Inference"):
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
        
#         outputs = inference_model(
#             input_ids=input_ids,
#             attention_mask=attention_mask
#         )
        
#         logits = outputs.logits
#         batch_predictions = torch.argmax(logits, dim=-1)
#         predictions.extend(batch_predictions.cpu().numpy())

# print(f"✅ 추론 완료: {len(predictions):,}개 예측")


In [ ]:
# # 제출 파일 생성
# print("제출 파일 생성 중...")

# # 예측 결과를 데이터프레임에 추가
# test_df["pred"] = predictions

# # 클래스별 예측 분포 확인
# unique_predictions, counts = np.unique(predictions, return_counts=True)
# print("\n클래스별 예측 분포:")
# for pred, count in zip(unique_predictions, counts):
#     percentage = (count / len(predictions)) * 100
#     class_name = LABEL_MAPPING.get(pred, f"클래스 {pred}")
#     print(f"   {class_name} ({pred}): {count:,}개 ({percentage:.1f}%)")

# # 샘플 제출 파일 로드
# sample_submission = pd.read_csv("data/sample_submission.csv")

# # ID를 기준으로 병합하여 제출 파일 생성
# submission_df = sample_submission[["ID"]].merge(
#     test_df[["ID", "pred"]], 
#     left_on="ID", 
#     right_on="ID", 
#     how="left"
# )

# # 제출 파일 검증
# assert len(submission_df) == len(sample_submission), f"길이 불일치: {len(submission_df)} vs {len(sample_submission)}"
# assert submission_df["pred"].isin([0, 1, 2, 3]).all(), "모든 예측값은 [0, 1, 2, 3] 범위에 있어야 합니다"
# assert not submission_df["pred"].isnull().any(), "예측값에 null 값이 있으면 안됩니다"
# assert not submission_df["ID"].isnull().any(), "ID 컬럼에 null 값이 있으면 안됩니다"

# print("✅ 제출 파일 검증 통과")

# # 제출 파일 저장
# submission_path = "./output.csv"
# submission_df.to_csv(submission_path, index=False)
# print(f"✅ 제출 파일 저장 완료: {submission_path}")

# # WandB 종료
# if config.use_wandb:
#     wandb.finish()
#     print("✅ WandB 세션 종료")

# print("\n🎉 모든 작업 완료!")


In [ ]:
# # 테스트 데이터 추론
# print("=" * 50)
# print("테스트 데이터 추론")
# print("=" * 50)

# # 테스트 데이터 로드
# test_df = pd.read_csv("data/test.csv")
# print(f"테스트 데이터: {len(test_df):,}개")

# # 최고 성능 모델 로드
# if os.path.exists("./best_model"):
#     print("최고 성능 모델 로드 중...")
#     inference_model = AutoModelForSequenceClassification.from_pretrained("./best_model")
#     inference_tokenizer = AutoTokenizer.from_pretrained("./best_model")
#     inference_model = inference_model.to(device)
#     inference_model.eval()
#     print("✅ 최고 성능 모델 로드 완료")
# else:
#     print("⚠️  저장된 모델이 없습니다. 현재 모델 사용")
#     inference_model = model
#     inference_tokenizer = tokenizer

# # 테스트 데이터셋 생성
# test_dataset = ReviewDataset(
#     test_df["review"],
#     None,  # 테스트 데이터는 라벨 없음
#     inference_tokenizer,
#     config.max_length,
# )

# # 테스트 데이터로더 생성
# test_dataloader = DataLoader(
#     test_dataset,
#     batch_size=config.eval_batch_size,
#     shuffle=False,
#     num_workers=2,
#     pin_memory=True if torch.cuda.is_available() else False
# )

# print("✅ 테스트 데이터셋 준비 완료")
